<a href="https://colab.research.google.com/github/yashrakeshmishra/Coursera_Capstone/blob/master/capstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project IBM Data Science Course

This Notebook is for the final Capstone Project in the IBM Data Science Professional Certificate course. It will involve a series of operations as required by the project.

## Predicting the severity of accident based on different conditions.

The model is trained using a classification algorithm. A data set consisting of road accidents in the city of Seattle from 2014 to present is used to train the model and evaluate it's metrics. Since the dataset only contains about two types of severities; namely slight and severe. A classic binary classfication algorithm is used to construct a model.

### Importing the important libraries for handling data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Downloading the data from the internet and storing it as a Pandas dataframe.

In [ ]:
df = pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')

### Refining the data

Important colums which will we required to evaluate the severity are kept and other non essential elements are dropped.

In [ ]:
data = df[['SEVERITYCODE','PERSONCOUNT','VEHCOUNT','WEATHER','ROADCOND','LIGHTCOND','INATTENTIONIND','UNDERINFL']]
attributes = data.columns.to_list()

The data obtained has many null values. We could have eliminated the null values but that would result in loss of dataset. Therefore, we replace the NaN values with the value with highest frequency in th column, since that value has the highest probability.

In [ ]:
for column in attributes:
    data[column].fillna(data[column].mode()[0], inplace=True)

The columns <code> INATTENTIONIND </code> and <code> UNDERINFL </code>contained mixed values such as 'Y','N',1,0. Therefore we convert the data into numerical form.

In [ ]:
data['INATTENTIONIND'].replace('Y',1,inplace=True)
data['INATTENTIONIND'].replace('N',0,inplace=True)
data.INATTENTIONIND.astype('int64')
data['UNDERINFL'].replace('Y',1,inplace=True)
data['UNDERINFL'].replace('N',0,inplace=True)
data.UNDERINFL.astype('int64');


Using the One-Hot-Encoding for the weather followed by dropping the weather column to obtain additonal columns with different types of weathers. The column value for a particular weather is 1 or 0 depending if the weather condition was true at the time of that particular accident.

In [ ]:
weathers=pd.get_dummies(data.WEATHER)
data = pd.concat([data,weathers],axis=1)

In [ ]:
data.drop(columns='WEATHER', inplace=True)

In [ ]:
data.rename(columns={'Fog/Smog/Smoke': 'Smoke','Sleet/Hail/Freezing Rain':'Hail', 'Unknown': 'Unpredictable', 'Blowing Sand/Dirt': 'Sandy'}, inplace=True)

In [ ]:
data.head()

,SEVERITYCODE,PERSONCOUNT,VEHCOUNT,ROADCOND,LIGHTCOND,INATTENTIONIND,UNDERINFL,Sandy,Clear,Smoke,Other,Overcast,Partly Cloudy,Raining,Severe Crosswind,Hail,Snowing,Unpredictable
0,2,2,2,Wet,Daylight,1,0,0,0,0,0,1,0,0,0,0,0,0
1,1,2,2,Wet,Dark - Street Lights On,1,0,0,0,0,0,0,0,1,0,0,0,0
2,1,4,3,Dry,Daylight,1,0,0,0,0,0,1,0,0,0,0,0,0
3,1,3,3,Dry,Daylight,1,0,0,1,0,0,0,0,0,0,0,0,0
4,2,2,2,Wet,Daylight,1,0,0,0,0,0,0,0,1,0,0,0,0
